In [1]:
!python -m build
%pip install dist/ichigo-0.0.1-py3-none-any.whl

* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools>=61.0
* Getting build dependencies for sdist...
running egg_info
writing ichigo/ichigo.egg-info/PKG-INFO
writing dependency_links to ichigo/ichigo.egg-info/dependency_links.txt
writing requirements to ichigo/ichigo.egg-info/requires.txt
writing top-level names to ichigo/ichigo.egg-info/top_level.txt
reading manifest file 'ichigo/ichigo.egg-info/SOURCES.txt'
writing manifest file 'ichigo/ichigo.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing ichigo/ichigo.egg-info/PKG-INFO
writing dependency_links to ichigo/ichigo.egg-info/dependency_links.txt
writing requirements to ichigo/ichigo.egg-info/requires.txt
writing top-level names to ichigo/ichigo.egg-info/top_level.txt
reading manifest file 'ichigo/ichigo.egg-info/SOURCES.txt'
writing manifest file 'ichigo/ichigo.egg-info/SOURCES.txt'
running check
creating ichigo-0.0.1
creating ichigo-0.0.1/ichigo/asr


## ASR

### Batch Processing

1. Transcribe with your audio file

In [3]:
# Quick one-liner
from ichigo.asr import transcribe
results = transcribe("test/speech.wav")
results

/opt/conda/envs/ichigo-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('Doanh nghiệp này vừa thay đổi đăng ký kinh doanh cuối tháng 9 năm 2023 có vốn điều lệ hơn 2.000 tỷ đồng, trong đó bà Thành nắm 77,15%, ông Khoa nắm 22,77%, và ông Quã nắm 0,08%.',
 {'duration': 14.6986875,
  'process_time': 3.1061782836914062,
  'rtf': 0.2113235133199074})

In [4]:
from ichigo.asr import IchigoASR
model = IchigoASR(model_name="merge-medium-vi-2d-2560c-dim64", 
                  model_path="homebrewltd/ichigo-whisper:merge-medium-vi-2d-2560c-dim64.pth")
results = model.transcribe(
    "test/folder",
    output_path="test/transcriptions",
    extensions=(".wav", ".mp3", ".flac", ".m4a")
)
results

Successfully transcribed: speech copy.wav
Successfully transcribed: speech.wav

Transcription Summary:
- Total files in directory: 2
- Audio files processed: 2
- Non-audio files skipped: 0
- Successful transcriptions: 2
- Failed transcriptions: 0


### API

```bash
# Start the API server (terminal)
uvicorn ichigo.asr.server:app --host 0.0.0.0 --port 8000

# Use with curl
curl -X POST "http://localhost:8000/transcribe/" \
  -H "accept: application/json" \
  -H "Content-Type: multipart/form-data" \
  -F "file=@path/to/audio.wav"

# Get semantic tokens
curl -X POST "http://localhost:8000/transcribe/?return_stoks=true" \
  -H "accept: application/json" \
  -H "Content-Type: multipart/form-data" \
  -F "file=@path/to/audio.wav"
```

You can also access the API documentation at `http://localhost:8000/docs`

## LLMs

### Python Usage

In [1]:
# Ichigo <= v0.4
from ichigo.llm import process_audio
response = process_audio("test/speech.wav")
response

/opt/conda/envs/ichigo-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.14it/s]
Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"I'm sorry, but I can't provide the information you're looking for because you didn't specify the name of the show. Could you please provide the name of the show you're referring to?"

In [1]:
# Or with more control using the assistant class
from ichigo.llm import IchigoAssistant
# Use default Ichigo model
assistant = IchigoAssistant()
response = assistant.generate_text(
    "test/speech.wav",
    max_new_tokens=2048
)
response

/opt/conda/envs/ichigo-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.02s/it]
Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"I'm sorry, but I can't provide the information you're looking for because you didn't specify the name of the show. Could you please provide the name of the show you're referring to?"

In [1]:
# Speechless model
from ichigo.llm import IchigoAssistant
# Or use Speechless model
assistant = IchigoAssistant(use_speechless=True)
response = assistant.generate_text(
    "test/speech.wav",
    max_new_tokens=2048
)
response

/opt/conda/envs/ichigo-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.02it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Dựa trên thông tin bạn cung cấp, tôi có thể tóm tắt tình hình tài chính của doanh nghiệp như sau:\n\n- Doanh nghiệp đã thay đổi đăng ký kinh doanh vào tháng 9 năm 2023.\n- Bà Thành nắm giữ 77,15% cổ phần.\n- Ông Khoa nắm giữ 22,77% cổ phần.\n- Ông Quan nắm giữ 0,08% cổ phần.\n\nTuy nhiên, để có cái nhìn toàn diện hơn về tình hình tài chính của doanh nghiệp, chúng ta cần thêm nhiều thông tin khác như:\n\n1. Tổng số cổ phần của doanh nghiệp.\n2. Số lượng cổ phần mà mỗi người nắm giữ.\n3. Giá trị của mỗi cổ phần.\n4. Các cổ đông khác có thể có bao nhiêu cổ phần.\n5. Mục đích của việc thay đổi đăng ký kinh doanh.\n6. Thời gian hoạt động của doanh nghiệp.\n\nNếu bạn có thêm thông tin chi tiết, tôi sẽ có thể phân tích sâu hơn về tình hình tài chính của doanh nghiệp này.'

### API

```bash
# Start the API server
uvicorn ichigo.llm.server:app --host 0.0.0.0 --port 8001

# Use with default Ichigo model
curl -X POST "http://localhost:8001/chat/" \
  -H "accept: application/json" \
  -H "Content-Type: multipart/form-data" \
  -F "file=@path/to/audio.wav"

# Use with Speechless model
curl -X POST "http://localhost:8001/chat/?use_speechless=true" \
  -H "accept: application/json" \
  -H "Content-Type: multipart/form-data" \
  -F "file=@path/to/audio.wav"
```

You can also access the API documentation at `http://localhost:8001/docs`